In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
from pathlib import Path
import xarray as xr
import xclimate as xclim

CESM2_COMPONENT_MAP = {
    "atm": "cam",
    "lnd": "clm2",
    "cice": "cice",
    "mosart": "rof",
    "glc": "cism",
    "cpl": "cpl",
}

In [6]:
ds = xclim.load_cesm2le("EFLX_LH_TOT", "lnd", "month_1", "h0", keep_var_only=False)

In [7]:
ds

<xarray.Dataset> Size: 15GB
Dimensions:      (member: 5, time: 1980, lat: 192, lon: 288, hist_interval: 2,
                  levgrnd: 25, levlak: 10, levdcmp: 25)
Coordinates:
  * levgrnd      (levgrnd) float32 100B 0.01 0.04 0.09 0.16 ... 19.48 28.87 42.0
  * levlak       (levlak) float32 40B 0.05 0.6 2.1 4.6 ... 18.6 25.6 34.33 44.78
  * levdcmp      (levdcmp) float32 100B 0.01 0.04 0.09 0.16 ... 19.48 28.87 42.0
  * lon          (lon) float32 1kB 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
  * lat          (lat) float32 768B -90.0 -89.06 -88.12 ... 88.12 89.06 90.0
  * time         (time) object 16kB 1850-02-01 00:00:00 ... 2015-01-01 00:00:00
  * member       (member) int64 40B 1 2 3 4 5
Dimensions without coordinates: hist_interval
Data variables:
    area         (member, time, lat, lon) float32 2GB dask.array<chunksize=(1, 120, 192, 288), meta=np.ndarray>
    landfrac     (member, time, lat, lon) float32 2GB dask.array<chunksize=(1, 120, 192, 288), meta=np.ndarray>
    landmask     (member, time, lat, lon) float64 4GB dask.array<chunksize=(1, 120, 192, 288), meta=np.ndarray>
    pftmask      (member, time, lat, lon) float64 4GB dask.array<chunksize=(1, 120, 192, 288), meta=np.ndarray>
    mcdate       (member, time) int32 40kB dask.array<chunksize=(1, 120), meta=np.ndarray>
    mcsec        (member, time) int32 40kB dask.array<chunksize=(1, 120), meta=np.ndarray>
    mdcur        (member, time) int32 40kB dask.array<chunksize=(1, 120), meta=np.ndarray>
    mscur        (member, time) int32 40kB dask.array<chunksize=(1, 120), meta=np.ndarray>
    nstep        (member, time) int32 40kB dask.array<chunksize=(1, 120), meta=np.ndarray>
    time_bounds  (member, time, hist_interval) object 158kB dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    EFLX_LH_TOT  (member, time, lat, lon) float32 2GB dask.array<chunksize=(1, 1, 192, 288), meta=np.ndarray>
Attributes: (12/100)
    title:                                     CLM History file information
    comment:                                   NOTE: None of the variables ar...
    Conventions:                               CF-1.0
    history:                                   created on 03/04/21 21:56:44
    source:                                    Community Land Model CLM4.0
    hostname:                                  aleph
    ...                                        ...
    cft_irrigated_switchgrass:                 60
    cft_tropical_corn:                         61
    cft_irrigated_tropical_corn:               62
    cft_tropical_soybean:                      63
    cft_irrigated_tropical_soybean:            64
    time_period_freq:                          month_1

In [ ]:
def load_cesm2le(
    variable: str | list[str],
    gcomp: str,
    frequency: str,
    stream: str,
    experiment: str = "historical",
    drop_vars: tuple = ("gw", "hyam", "hybm", "P0", "ilev", "slat", "slon", "w_stag"),
    keep_cosp: bool = False,
    keep_var_only: bool = False,
    chunk: bool | dict = False,
) -> xr.Dataset:
    """
    Load variables from the CESM2 Large Ensemble.

    Parameters
    ----------
    variable : str or list of str
        The name(s) of the variable(s) to load.
    gcomp : str
        The component of the model the variable belongs to (e.g., "atm", "lnd").
    frequency : str
        The frequency of the output (e.g., "month_1").
    experiment : str, optional
        The experiment to load data from. Can be "historical" or "ssp370".
        Defaults to "historical".
    drop_vars : list or tuple, optional
        A list of variables to drop from the dataset.
        Defaults to ('gw', 'hyam', 'hybm', 'P0', 'ilev', 'slat', 'slon', 'w_stag').
    keep_cosp : bool, optional
        If False, remove all COSP-related variables and coordinates.
        Defaults to False.
    keep_var_only : bool, optional
        If True, only keep the specified variable and its coordinates.
        Defaults to False.
    chunk : bool or dict, optional
        If True, rechunk the dataset to a default chunking scheme.
        If a dict, rechunk the dataset to the specified chunk sizes.
        Defaults to False.

    Returns
    -------
    xr.Dataset
        A dataset containing the loaded data, with an added "member" dimension.
    """
    if isinstance(variable, str):
        variables = [variable]
    else:
        variables = variable

    scomp = CESM2_COMPONENT_MAP[gcomp]
    exp_dict = {
        "historical": "BHISTsmbb",
        "ssp370": "BSSP370smbb",
    }

    rootdir = Path("/glade/campaign/collections/gdex/data/d651056/CESM2-LE")

    member_datasets = []
    for m in range(1, 101):
        # Determine member_id
        if experiment == "historical":
            member_id = f"10{str(m-1).zfill(2)}"
        elif experiment == "ssp370":
            member_id = (
                f"12{str(30+m).zfill(2)}" if m <= 70 else f"13{str(m-71).zfill(2)}"
            )
        else:
            continue

        variable_datasets = []
        for var_name in variables:
            tsdir = rootdir / f"{gcomp}/proc/tseries/{frequency}/{var_name}"
            file_pattern = f"b.e21.{exp_dict[experiment]}.f09_g17.LE2-{member_id}.*.{scomp}.*.{var_name}.*.nc"
            files = sorted(list(tsdir.glob(file_pattern)))
            if files:
                ds = xr.open_mfdataset(files, decode_timedelta=False)
                variable_datasets.append(ds)

        if variable_datasets:
            # Merge variables for the current member
            member_ds = xr.merge(variable_datasets)
            member_datasets.append(member_ds)

    if not member_datasets:
        return xr.Dataset()

    # Concatenate all member datasets
    combined_ds = xr.concat(member_datasets, dim="member")
    combined_ds = combined_ds.assign_coords(member=range(1, len(member_datasets) + 1))

    if drop_vars:
        existing_vars_to_drop = [
            var
            for var in drop_vars
            if var in combined_ds.coords or var in combined_ds.data_vars
        ]
        if existing_vars_to_drop:
            combined_ds = combined_ds.drop_vars(existing_vars_to_drop)

    if not keep_cosp:
        cosp_vars = [var for var in combined_ds.variables if "cosp" in var]
        if cosp_vars:
            combined_ds = combined_ds.drop_vars(cosp_vars)

    if keep_var_only:
        # Ensure we only keep variables that were actually found and loaded
        found_variables = [v for v in variables if v in combined_ds.data_vars]
        combined_ds = combined_ds[found_variables]

    if chunk:
        if isinstance(chunk, dict):
            combined_ds = combined_ds.chunk(chunk)
        else:
            # Default rechunking: chunk time dimension fully
            combined_ds = combined_ds.chunk({"time": -1})

    return combined_ds

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3197457646.py, line 5)

In [ ]:
variable = ["EFLX_LH_TOT"]
gcomp = "lnd"
frequency = "month_1"
stream = "h0"
experiment = "historical"
drop_vars = ("gw", "hyam", "hybm", "P0", "ilev", "slat", "slon", "w_stag")
keep_cosp = False
keep_var_only = True
chunk = False

if isinstance(variable, str):
    variables = [variable]
else:
    variables = variable

scomp = CESM2_COMPONENT_MAP[gcomp]
exp_dict = {
    "historical": "BHISTsmbb",
    "ssp370": "BSSP370smbb",
}

rootdir = Path("/glade/campaign/collections/gdex/data/d651056/CESM2-LE")

member_datasets = []
for m in range(1, 101):
    # Determine member_id
    if experiment == "historical":
        member_id = f"10{str(m-1).zfill(2)}"
    elif experiment == "ssp370":
        member_id = (
            f"12{str(30+m).zfill(2)}" if m <= 70 else f"13{str(m-71).zfill(2)}"
        )
    else:
        continue

    variable_datasets = []
    for var_name in variables:
        tsdir = rootdir / f"{gcomp}/proc/tseries/{frequency}/{var_name}"
        file_pattern = f"b.e21.{exp_dict[experiment]}.f09_g17.LE2-{member_id}.*.{scomp}.{stream}.{var_name}.*.nc"
        files = sorted(list(tsdir.glob(file_pattern)))
        if files:
            ds = xr.open_mfdataset(files, decode_timedelta=False, drop_variables=("time_written", "date_written", "test"))
            variable_datasets.append(ds)

    if variable_datasets:
        # Merge variables for the current member
        member_ds = xr.merge(variable_datasets)
        member_datasets.append(member_ds)

# Concatenate all member datasets
combined_ds = xr.concat(member_datasets, dim="member")
combined_ds = combined_ds.assign_coords(member=range(1, len(member_datasets) + 1))

if drop_vars:
    existing_vars_to_drop = [
        var
        for var in drop_vars
        if var in combined_ds.coords or var in combined_ds.data_vars
    ]
    if existing_vars_to_drop:
        combined_ds = combined_ds.drop_vars(existing_vars_to_drop)

if not keep_cosp:
    cosp_vars = [var for var in combined_ds.variables if "cosp" in var]
    if cosp_vars:
        combined_ds = combined_ds.drop_vars(cosp_vars)

if keep_var_only:
    # Ensure we only keep variables that were actually found and loaded
    found_variables = [v for v in variables if v in combined_ds.data_vars]
    combined_ds = combined_ds[found_variables]

if chunk:
    if isinstance(chunk, dict):
        combined_ds = combined_ds.chunk(chunk)
    else:
        # Default rechunking: chunk time dimension fully
        combined_ds = combined_ds.chunk({"time": -1})

/glade/u/home/bbuchovecky/miniforge3/envs/cpl_ppe_co2/lib/python3.12/site-packages/xarray/core/indexing.py:1680: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/glade/u/home/bbuchovecky/miniforge3/envs/cpl_ppe_co2/lib/python3.12/site-packages/xarray/core/indexing.py:1680: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     

In [19]:
combined_ds

<xarray.Dataset> Size: 392GB
Dimensions:      (member: 5, time: 1980, lat: 192, lon: 288, levgrnd: 25,
                  levlak: 10, hist_interval: 2, levdcmp: 25)
Coordinates:
  * levgrnd      (levgrnd) float32 100B 0.01 0.04 0.09 0.16 ... 19.48 28.87 42.0
  * levlak       (levlak) float32 40B 0.05 0.6 2.1 4.6 ... 18.6 25.6 34.33 44.78
  * levdcmp      (levdcmp) float32 100B 0.01 0.04 0.09 0.16 ... 19.48 28.87 42.0
  * lon          (lon) float32 1kB 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
  * lat          (lat) float32 768B -90.0 -89.06 -88.12 ... 88.12 89.06 90.0
  * time         (time) object 16kB 1850-02-01 00:00:00 ... 2015-01-01 00:00:00
  * member       (member) int64 40B 1 2 3 4 5
Dimensions without coordinates: hist_interval
Data variables: (12/20)
    area         (member, time, lat, lon) float32 2GB dask.array<chunksize=(1, 120, 192, 288), meta=np.ndarray>
    landfrac     (member, time, lat, lon) float32 2GB dask.array<chunksize=(1, 120, 192, 288), meta=np.ndarray>
    landmask     (member, time, lat, lon) float64 4GB dask.array<chunksize=(1, 120, 192, 288), meta=np.ndarray>
    pftmask      (member, time, lat, lon) float64 4GB dask.array<chunksize=(1, 120, 192, 288), meta=np.ndarray>
    nbedrock     (member, time, lat, lon) float64 4GB dask.array<chunksize=(1, 120, 192, 288), meta=np.ndarray>
    ZSOI         (member, time, levgrnd, lat, lon) float32 55GB dask.array<chunksize=(1, 120, 13, 96, 144), meta=np.ndarray>
    ...           ...
    mcsec        (member, time) float64 79kB dask.array<chunksize=(1, 120), meta=np.ndarray>
    mdcur        (member, time) float64 79kB dask.array<chunksize=(1, 120), meta=np.ndarray>
    mscur        (member, time) float64 79kB dask.array<chunksize=(1, 120), meta=np.ndarray>
    nstep        (member, time) float64 79kB dask.array<chunksize=(1, 120), meta=np.ndarray>
    time_bounds  (member, time, hist_interval) object 158kB dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    EFLX_LH_TOT  (member, time, lat, lon) float32 2GB dask.array<chunksize=(1, 1, 192, 288), meta=np.ndarray>
Attributes: (12/100)
    title:                                     CLM History file information
    comment:                                   NOTE: None of the variables ar...
    Conventions:                               CF-1.0
    history:                                   created on 03/04/21 21:56:44
    source:                                    Community Land Model CLM4.0
    hostname:                                  aleph
    ...                                        ...
    cft_irrigated_switchgrass:                 60
    cft_tropical_corn:                         61
    cft_irrigated_tropical_corn:               62
    cft_tropical_soybean:                      63
    cft_irrigated_tropical_soybean:            64
    time_period_freq:                          month_1